### Run the cell below and update the Mongo Host and Port Info

In [20]:
# Run the cell below to get the mongo public port, then 
# go to `packages/syft/src/syft/core/node/new/mongo_client.py` and 
# change MONGO_HOST = "localhost" and
# MONGO_PORT = "port_number_shown_below"

! bash ~/PySyft/scripts/dev_tools.sh

-----------------------------
Containers | Exposed Ports
-----------------------------
/canada-seaweedfs-1  49161 49160 
/canada-mongo-1  49155 
/canada-queue-1  49156 
/canada-jaeger-1  14268 49154 
/canada-redis-1  49153 


In [ ]:
## Start here.

In [1]:
import syft as sy
from syft.core.node.new.user import UserCreate, User, UserUpdate
from syft.core.node.new.mongo_document_store import MongoCollection, MongoDocumentStore, MongoBsonObject
from syft.core.node.common.node_table.syft_object import SyftObjectRegistry
from syft.core.node.worker import Worker
from syft.core.node.new.user_stash import UserStash
from syft.core.common.uid import UID

kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /home/shubham/PySyft


In [2]:
from syft.core.node.new.document_store import UIDPartitionKey

In [3]:
store = MongoDocumentStore()

In [4]:
mongo_store_ser = sy.serialize(store, to_bytes=True)

In [5]:
store = sy.deserialize(mongo_store_ser, from_bytes=True)

In [6]:
user_stash = UserStash(store=store)

In [21]:
user_stash.partition.db_collection.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)]},
 'User_index_name': {'v': 2,
  'key': [('email', 1), ('signing_key', 1), ('verify_key', 1)],
  'unique': True}}

In [7]:
new_user = UserCreate(
    name="Shubham", email="alice@openmined.org", password="pass", password_verify="pass"
)

In [8]:
user = new_user.to(User)

In [13]:
bson_dict = user.to(MongoBsonObject)

In [14]:
u = MongoBsonObject.to_syft_obj(bson_dict, User)

In [15]:
response = user_stash.set(user)

In [16]:
response

Err('Duplicate Key Error: E11000 duplicate key error collection: app.User index: User_index_name dup key: { email: "alice@openmined.org", signing_key: null, verify_key: null }, full error: {\'index\': 0, \'code\': 11000, \'errmsg\': \'E11000 duplicate key error collection: app.User index: User_index_name dup key: { email: "alice@openmined.org", signing_key: null, verify_key: null }\', \'keyPattern\': {\'email\': 1, \'signing_key\': 1, \'verify_key\': 1}, \'keyValue\': {\'email\': \'alice@openmined.org\', \'signing_key\': None, \'verify_key\': None}}')

In [22]:
# #### Create indexes
# from pymongo import IndexModel, ASCENDING, DESCENDING
# user_stash.partition.db_collection.create_index([
#     ("email", ASCENDING),
#     ("verify_key", ASCENDING)
# ], unique=True)

In [28]:
search_result = user_stash.get_by_email(email="alice@openmined.org")


In [30]:
search_result.ok() if search_result.is_ok() else search_result.err()

```python
class User:
  id: str = 68e8f4bc81a949e98e641ead22300ffb
  email: str = "alice@openmined.org"
  name: str = "Shubham"
  hashed_password: str = "$2b$12$vbK9bo6.iEarmFKwbE7zAOzEtW0jA26rCA2SG/6PfXmyX5TWXemOW"
  salt: str = "$2b$12$vbK9bo6.iEarmFKwbE7zAO"
  signing_key: str = 634dd9199241fbbc82ba844da019c2fdabadff1dc452ccb29e8f39af105a5cda
  verify_key: str = 9040173c7cb875e9fc5a047bcf792bf360255835a326274dbf1b6e5a4a9c451a
  role: str = ServiceRole.GUEST
  institution: str = None
  website: str = None
  created_at: str = None

```

In [31]:
uid_to_update = UID.from_string("68e8f4bc81a949e98e641ead22300ffb")

In [32]:
update_user = UserUpdate(id=uid_to_update, email="alice@openmined.org", name="MyBob2", institution="OpenMined")

In [33]:
update_result = user_stash.update(user=update_user.to(User))

In [35]:
update_result.ok() if update_result.is_ok() else update_result.err()

```python
class User:
  id: str = 68e8f4bc81a949e98e641ead22300ffb
  email: str = "alice@openmined.org"
  name: str = "MyBob2"
  hashed_password: str = None
  salt: str = None
  signing_key: str = None
  verify_key: str = None
  role: str = None
  institution: str = "OpenMined"
  website: str = None
  created_at: str = None

```

In [36]:
user_stash.get_by_email(email="bob@alice.com")

Ok(None)

In [39]:
uid_to_delete = UID.from_string("68e8f4bc81a949e98e641ead22300ffb")
response = user_stash.delete_by_uid(uid=uid_to_delete)

In [40]:
response

Ok(<class 'syft.core.node.new.response.SyftSuccess'>: ID: 68e8f4bc81a949e98e641ead22300ffb deleted)

#### Note: Change back the Mongo Host and port to default before running the cells below and restart kernel

`MONGO_HOST = "mongo"
MONGO_PORT = "27017"`

In [7]:
import syft as sy
from syft.core.node.new.user import UserSearch, UserCreate

In [2]:
client = sy.new_login(email="info@openmined.org", password="changethis", port=8081)

Logged into canada as <info@openmined.org>


In [8]:
u_create = UserCreate(name="Shubham", email="shubham@openmined.org", password="pass", password_verify="pass")

In [9]:
client.api.services.user.create(user_create=u_create)

```python
class UserView:
  id: str = 2cd283d109f1401385542e6d53af9c77
  email: str = "shubham@openmined.org"
  name: str = "Shubham"
  role: str = ServiceRole.GUEST
  password: str = None
  password_verify: str = None
  verify_key: str = None
  institution: str = None
  website: str = None

```

In [10]:
client.api.services.user.search(name="Shubham")

[class UserView:
   id: str = 2cd283d109f1401385542e6d53af9c77
   email: str = "shubham@openmined.org"
   name: str = "Shubham"
   role: str = ServiceRole.GUEST
   password: str = None
   password_verify: str = None
   verify_key: str = None
   institution: str = None
   website: str = None]

In [12]:
uid_to_delete = UID.from_string("2cd283d109f1401385542e6d53af9c77")
client.api.services.user.delete(uid_to_delete)

<class 'syft.core.node.new.response.SyftSuccess'>: ID: 2cd283d109f1401385542e6d53af9c77 deleted

In [13]:
client.api.services.user.search(name="Shubham")

[]